In [40]:
import pandas as pd
from pybliometrics.scopus import AffiliationRetrieval
from tqdm import tqdm

pd.set_option('display.max_columns', None)
data = pd.read_csv("./data/ai_research_papers.csv")
data.at[222653, 'author_names'] = 'Dudík, Miroslav;Haghtalab, Nika;Luo, Haipeng;Schapire, Robert E.;Syrgkanis, Vasilis;Vaughan, Jennifer Wortman'
data_exploded = data.assign(author_afids=data['author_afids'].str.split(';'), author_ids=data['author_ids'].str.split(';'), author_names = data['author_names'].str.split(';'))
data_exploded = data_exploded.explode(['author_afids', 'author_names', 'author_ids'])
data_exploded['author_rank'] = data_exploded.groupby('eid').cumcount() + 1

/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:365: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2,19,21,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [41]:
data_exploded[['eid', 'coverDate', 'origin_ref','author_count', 'author_names', 'author_ids', 'author_afids', 'author_rank', 'citedby_count']]

,eid,coverDate,origin_ref,author_count,author_names,author_ids,author_afids,author_rank,citedby_count
0,2-s2.0-0030354250,1996-12-01,AAAI Artificial Intelligence,2.0,"Baffes, Paul T.",6508335676,60092463,1,24
0,2-s2.0-0030354250,1996-12-01,AAAI Artificial Intelligence,2.0,"Mooney, Raymond J.",7102791999,60092463,2,24
1,2-s2.0-0029736257,1996-01-01,INFOCOM Networking,1.0,"Ramanathan, S.",7102449182,60011761,1,13
2,2-s2.0-85088074507,1996-01-01,OSDI Operating Systems,3.0,"Mowry, Tödd C.",7003819429,60016849,1,135
2,2-s2.0-85088074507,1996-01-01,OSDI Operating Systems,3.0,"Demke, Angela K.",57212549974,60016849,2,135
...,...,...,...,...,...,...,...,...,...
270862,2-s2.0-85146950429,2023-06-19,SIGMETRICS Performance,5.0,"Schmid, Stefan",35424598400,60011604,4,4
270862,2-s2.0-85146950429,2023-06-19,SIGMETRICS Performance,5.0,"Avin, Chen",8983767700,60027161,5,4
270863,2-s2.0-85146930099,2023-06-19,SIGMETRICS Performance,3.0,"Addanki, Vamsi",58578019200,60011604,1,5
270863,2-s2.0-85146930099,2023-06-19,SIGMETRICS Performance,3.0,"Avin, Chen",8983767700,60027161,2,5


In [42]:
from pybliometrics.scopus import AuthorRetrieval
au = AuthorRetrieval(6508335676)

In [45]:
au.citation_count
au.document_count
au.h_index
au.orcid
au.publication_range

import pandas as pd
areas = pd.DataFrame(au.subject_areas)
areas

,area,abbreviation,code
0,Aerospace Engineering,ENGI,2202
1,Computational Mathematics,MATH,2605
2,Electrical and Electronic Engineering,ENGI,2208
3,Computer Vision and Pattern Recognition,COMP,1707
4,Computer Science Applications,COMP,1706
5,Applied Mathematics,MATH,2604
6,"Electronic, Optical and Magnetic Materials",MATE,2504
7,Communication,SOCI,3315
8,Control and Systems Engineering,ENGI,2207
9,Computer Networks and Communications,COMP,1705


In [46]:

au.h_index

4

In [47]:
au.citation_count

77

In [48]:
au.document_count

9